In [1]:
logFileName = "car_log.txt"

In [2]:
class Logger:
    def __init__(self):
        self.cars = list()
    
    
    def addCar(self, car):
        self.cars.append(car)
    
    
    def outputToFile(self):
        f = open(logFileName, "w")
        
        for car in self.cars:
            locationStrList = list()
            for loc in car.location_log:
                locationStrList.append(str(loc[0]) + "," + str(loc[1]))
            
            line = str(car.id) + " "      
            line += " ".join(locationStrList) + "\n"
            f.write(line)
            
        f.close()

In [3]:
class Vehicle:
    types = ["Sedan", "Truck", "Motorcycle"]
    # Super-class of car, TBI (to be implemented)

In [4]:
class Car:
    max_speed = 10.0
    min_speed = 0
    max_acc = 0.5
    max_brake = 2
    size = 2.0
    
    
    def __init__(self, id, type, driver, initialSpeed=0.0, initialLocation=[0.0, 0.0]):
        self.id = id
        self.type = type
        self.speed = initialSpeed
        self.location = initialLocation
        self.location_log = list()
        self.logPosition()
        self.driver = driver
        self.combo = 0.0  # This variable is used to simulate smooth braking
        self.lane = None
        self.is_collided = False
        
        
    def accelerate(self):
        self.combo = 0  # Reset the brake combo
        self.speed += self.max_acc
        if self.speed > self.max_speed:
            self.speed = self.max_speed
        self.__move()
        
        
    def decelerate(self):
        dec_coef = 0.0
        combo_increment = 0.0
        dtype = self.driver.type
        if dtype == "Rookie":
            dec_coef = 0.9
            combo_increment = 0.8
        elif dtype == "Hasty":
            dec_coef = 0.1
            combo_increment = 0.05
        elif dtype == "Professional":
            dec_coef = 0.5
            combo_increment = 0.2
            
        speed_reduction = dec_coef * Car.max_brake + self.combo
        self.combo += combo_increment
        if speed_reduction > Car.max_brake:
            speed_reduction = Car.max_brake
        self.speed -= speed_reduction
        
        if self.speed < self.min_speed:
            self.speed = self.min_speed
            self.combo = 0
        self.__move()
        
    
    def __move(self):
        self.location[0] += self.speed
        self.logPosition()
    
    
    def search_car(self):
        for car in self.lane.carsOnLane.values():
            if car.id == self.id:
                continue
            self_front_bumper = self.location[0] + Car.size / 2
            car_back_bumper = car.location[0] - Car.size / 2
            return self_front_bumper < car_back_bumper <= self_front_bumper + self.driver.vision
                
                
    def set_lane(self, lane):
        self.lane = lane
        
        
    def logPosition(self):
        self.location_log.append(list(self.location))
        
        
#  It could be implemented as car + driver = agent
#  Also driver types should be classes like enums or interfaces instead strings?

In [5]:
class Driver:
    
    def __init__(self, type, gender, age):
        self.type = type
        self.gender = gender
        self.age = age
        self.vision = self.__set_vision()
        
        
    def __set_vision(self):
        avg_vision = 40
        age_threshold = 20
        return avg_vision + age_threshold - self.age
        
        
    # Adds more functionality to Car class objects or vice versa, TBI

In [6]:
class Lane:    
      
    def __init__(self, length, parentRoad):
        self.parentRoad = parentRoad
        self.length = length
        self.carsOnLane = {}
        self.carPositions = []
        
        
    def __str__(self):
        strprt = ""
        for car in self.carsOnLane:
            strprt += car + ":" + str(self.carsOnLane[car].location) + "  "
        return strprt
    
    
    
    def addToLane(self, car, pos):
        self.carsOnLane[car.id] = car
        self.carPositions.append(pos)
        car.set_lane(self)
        
       
    def isEmpty(self):
        return True if not self.carsOnLane else False

    
#  I think we need only one list containing car objects.
#  I converted the static variables to member variables

In [7]:
class Road:
    lanes = list()
    def __init__(self, laneCount):
        self.lanes = self.generateLanes(laneCount)
    
    @classmethod
    def generateLanes(self, laneCount):
        for i in range(0,laneCount):
            self.lanes.append(Lane(laneLength, self))

In [8]:
# SimulationEngine should take Road object instead of Lane as parameter, TBI
class SimulationEngine:
    def __init__(self, lane):
        self.lane = lane
        self.cars = lane.carsOnLane
        
    def moveCars(self):
        while(not self.lane.isEmpty()):
            for key in self.cars.copy():
                car = self.cars[key]
                if car.search_car():
                    car.decelerate()
                else:
                    car.accelerate()

                if(car.location[0] > self.lane.length):
                    self.lane.carsOnLane.pop(car.id)
            
            print(self.lane)

In [9]:
class LaneChanger:
    dummy_var = 0

In [10]:
# myRoad = Road(3)
# print(len(myRoad.lanes))

driver1 = Driver("Professional", "Female", 30)
driver2 = Driver("Rookie", "Male", 50)
myCar1 = Car("Sedan1", "Sedan", driver2, 4)
myCar2 = Car("Sedan2", "Sedan", driver1, 0, [30.0, 0.0])

myLane = Lane(200, "a")

#  I think we do not need the second parameter of addToLane method.
myLane.addToLane(myCar1, myCar1.location)
myLane.addToLane(myCar2, myCar2.location)

mySimEngine = SimulationEngine(myLane)
mySimEngine.moveCars()

myLogger = Logger()
myLogger.addCar(myCar1)
myLogger.addCar(myCar2)
myLogger.outputToFile()

Sedan1:[4.5, 0.0]  Sedan2:[30.5, 0.0]  
Sedan1:[9.5, 0.0]  Sedan2:[31.5, 0.0]  
Sedan1:[15.0, 0.0]  Sedan2:[33.0, 0.0]  
Sedan1:[21.0, 0.0]  Sedan2:[35.0, 0.0]  
Sedan1:[27.5, 0.0]  Sedan2:[37.5, 0.0]  
Sedan1:[32.2, 0.0]  Sedan2:[40.5, 0.0]  
Sedan1:[34.900000000000006, 0.0]  Sedan2:[44.0, 0.0]  
Sedan1:[35.60000000000001, 0.0]  Sedan2:[48.0, 0.0]  
Sedan1:[36.80000000000001, 0.0]  Sedan2:[52.5, 0.0]  
Sedan1:[38.500000000000014, 0.0]  Sedan2:[57.5, 0.0]  
Sedan1:[40.70000000000002, 0.0]  Sedan2:[63.0, 0.0]  
Sedan1:[43.40000000000002, 0.0]  Sedan2:[69.0, 0.0]  
Sedan1:[46.60000000000002, 0.0]  Sedan2:[75.5, 0.0]  
Sedan1:[50.300000000000026, 0.0]  Sedan2:[82.5, 0.0]  
Sedan1:[54.50000000000003, 0.0]  Sedan2:[90.0, 0.0]  
Sedan1:[59.20000000000003, 0.0]  Sedan2:[98.0, 0.0]  
Sedan1:[64.40000000000003, 0.0]  Sedan2:[106.5, 0.0]  
Sedan1:[70.10000000000004, 0.0]  Sedan2:[115.5, 0.0]  
Sedan1:[76.30000000000004, 0.0]  Sedan2:[125.0, 0.0]  
Sedan1:[83.00000000000004, 0.0]  Sedan2:[135.0, 